微调embedding模型以便增强RAG整体的检索能力

### 微调Embedding和微调LLM的关系

- 相似点

  两种类型的微调都遵循相同的方法，即生成用于训练和评估的数据集，微调模型，最后评估基本模型和微调模型之间的性能。
  使用LLM自动生成训练和评估数据集。

- 不同点

  数据集内容在LLM微调和Embedding模型微调之间有所不同。用于LLM微调的数据集包含LLM生成的问题。在微调过程中，包括问题、答案、系统prompt等在内的一系列数据将以JSON行( jsonl)文件的形式传递给要进行微调的模型。
  不同的是，用于Embedding模型微调的数据集包含以下三组:

  - `queries`：node_id映射和LLM生成的问题的集合。
  - `corpus`：node_id映射和相应节点中的文本的集合。
  - `relevant_docs`：查询的node_id和语料库 node_id之间的交叉引用映射的集合。给定一个查询，它告诉Embedding模型要查找哪个文本节点/语料库。

### 微调Embedding的具体步骤


主要任务包括:

1. 通过调用 `EmbeddingQAFinetuneDataset`函数`generate_qa_embedding_pairs`，自动生成评估和训练数据集的数据。
2. 通过传入基本模型和训练数据集来构造`SentenceTransformersFinetuneEngine`，然后调用其`finetune`函数来训练基本模型。
3. 创建经过微调的模型。
4. 调用向量存储索引检索器检索相关节点并评估基本模型的命中率。
5. 调用`InformationRetrievalEvaluator`来评估基本模型。
6. 调用向量存储索引检索器检索相关节点并评估微调模型的命中率。
7. 调用InformationRetrievalEvaluator来评估经过微调的模型。

In [ ]:
!pip install sentence-transformers pandas llama_index openai trulens_eval pypdf

In [18]:
from llama_index import SimpleDirectoryReader
from llama_index.node_parser import SimpleNodeParser, SentenceSplitter, SentenceWindowNodeParser
from llama_index.schema import MetadataMode
from llama_index.finetuning import generate_qa_embedding_pairs, EmbeddingQAFinetuneDataset
import openai

#### Step 1: 生成数据集

使用LLM来自动生成训练和评估的数据集。

In [5]:
# 下载数据集
!curl https://d18rn0p25nwr6d.cloudfront.net/CIK-0001045810/4e9abe7b-fdc7-4cd2-8487-dc3a99f30e98.pdf --output nvidia-sec-10k-2022.pdf

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1541k  100 1541k    0     0  4784k      0 --:--:-- --:--:-- --:--:-- 4786k


In [15]:
def load_corpus(docs, for_training=False, verbose=False):
    parser = SimpleNodeParser.from_defaults()
    if for_training:
        nodes = parser.get_nodes_from_documents(docs[:90], show_progress=verbose)
    else:
        nodes = parser.get_nodes_from_documents(docs[91:], show_progress=verbose)

    if verbose:
        print(f'Parsed {len(nodes)} nodes')

    return nodes

In [16]:
SEC_FILE = ['nvidia-sec-10k-2022.pdf']

print(f"Loading files {SEC_FILE}")

reader = SimpleDirectoryReader(input_files=SEC_FILE)
docs = reader.load_data()
print(f'Loaded {len(docs)} docs')

train_nodes = load_corpus(docs, for_training=True, verbose=True)
val_nodes = load_corpus(docs, for_training=False, verbose=True)

Loading files ['nvidia-sec-10k-2022.pdf']
Loaded 169 docs


Parsing nodes:   0%|          | 0/90 [00:00<?, ?it/s]

Parsed 97 nodes


Parsing nodes:   0%|          | 0/78 [00:00<?, ?it/s]

Parsed 84 nodes


In [17]:
docs[0]

Document(id_='04e690ba-e5b1-42ed-a9d9-b9d65dd4248d', embedding=None, metadata={'page_label': '1', 'file_name': 'nvidia-sec-10k-2022.pdf', 'file_path': 'nvidia-sec-10k-2022.pdf', 'file_type': 'application/pdf', 'file_size': 1578179, 'creation_date': '2023-12-21', 'last_modified_date': '2023-12-21', 'last_accessed_date': '2023-12-21'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, hash='3028d16923d6143a5d4afc8822380b577f4db603b52e9e89e8f01186832a4157', text='Table of Contents\nUNITED ST ATES\nSECURITIES AND EXCHANGE COMMISSION\nWashington, D.C. 20549\n____________________________________________________________________________________________\nFORM 10-K\n☒ ANNUAL  REPORT PURSUANT T O SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\n    For the fisca

In [19]:
# 生成合成查询和数据集
from llama_index.finetuning import (
    generate_qa_embedding_pairs,
    EmbeddingQAFinetuneDataset,
)
from llama_index.llms import OpenAI
from google.colab import userdata

openai.api_key = userdata.get('OPENAI_API_KEY')# os.environ["OPENAI_API_KEY"]

In [ ]:
# default using GPT-3.5-Turbo
DEFAULT_OPENAI_MODEL = "gpt-3.5-turbo"
llm = OpenAI(model=DEFAULT_OPENAI_MODEL)

train_dataset = generate_qa_embedding_pairs(train_nodes, llm=llm)
val_dataset = generate_qa_embedding_pairs(val_nodes, llm=llm)

train_dataset.save_json("train_dataset.json")
val_dataset.save_json("val_dataset.json")

In [24]:
train_dataset = EmbeddingQAFinetuneDataset.from_json("train_dataset.json")
val_dataset = EmbeddingQAFinetuneDataset.from_json("val_dataset.json")

#### Step 2: 微调Embedding模型

`SentenceTransformersFinetuneEngine`就是为这个任务设计的。在底层，它执行多个子任务:

- 通过构建SentenceTransformer加载预训练模型，传入BAAI/big-small-en模型id。
- 定义数据加载器。它加载我们的训练数据集，将其解析为查询，语料库和relevant_docs。然后循环查询，将relevant_docs中的node_id与corpus中的文本节点进行映射，构造InputExample，其列表依次传递到创建DataLoader中.
- 定义loss（损失函数）。它使用sentence_transformers multiplenegativerankingloss来训练检索设置的Embeddings。
- 定义评估器。它设置了一个带有eval数据集的评估器来监控Embedding模型在训练期间的表现。
- 运行训练。它插入上面定义的数据加载器、损失函数和评估器来运行训练。

LlamaIndex将微调Embedding模型的所有详细子任务封装在一个`SentenceTransformersFinetuneEngine`中，我们所需要做的就是调用它的`finetune`函数

In [25]:
from llama_index.finetuning import SentenceTransformersFinetuneEngine

finetune_engine = SentenceTransformersFinetuneEngine(
    train_dataset,
    model_id="BAAI/bge-small-en",
    model_output_path="test_model",
    val_dataset=val_dataset,
)

finetune_engine.finetune()

embed_model = finetune_engine.get_finetuned_model()

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.8k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/20 [00:00<?, ?it/s]

Iteration:   0%|          | 0/20 [00:00<?, ?it/s]

In [26]:
embed_model

HuggingFaceEmbedding(model_name='test_model', embed_batch_size=10, callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x7a389e160bb0>, tokenizer_name='test_model', max_length=512, pooling=<Pooling.CLS: 'cls'>, normalize=True, query_instruction=None, text_instruction=None, cache_folder=None)

#### Step 3: 评估微调后的模型

我们使用两种不同的评估方法:

- 命中率:对每个query / relevant_doc对进行简单的top-k检索。如果搜索结果包含relevant_doc，那么它就是一个“命中”。这可以用于专有的Embeddings，例如OpenAI的Embedding模型和开源Embedding模型。请参阅下面代码片段中的evaluate函数。

- `InformationRetrievalEvaluator`:一个更全面的用于评估开源Embeddings的度量套件。请参阅下面代码片段中的evaluate_st函数。

In [28]:
from llama_index.embeddings import OpenAIEmbedding
from llama_index import ServiceContext, VectorStoreIndex
from llama_index.schema import TextNode
from tqdm.notebook import tqdm
import pandas as pd

# function for hit rate evals
def evaluate(
    dataset,
    embed_model,
    top_k=5,
    verbose=False,
):
    corpus = dataset.corpus
    queries = dataset.queries
    relevant_docs = dataset.relevant_docs

    service_context = ServiceContext.from_defaults(embed_model=embed_model)
    nodes = [TextNode(id_=id_, text=text) for id_, text in corpus.items()]
    index = VectorStoreIndex(nodes, service_context=service_context, show_progress=True)
    retriever = index.as_retriever(similarity_top_k=top_k)

    eval_results = []
    for query_id, query in tqdm(queries.items()):
        retrieved_nodes = retriever.retrieve(query)
        retrieved_ids = [node.node.node_id for node in retrieved_nodes]
        expected_id = relevant_docs[query_id][0]
        is_hit = expected_id in retrieved_ids  # assume 1 relevant doc

        eval_result = {
            "is_hit": is_hit,
            "retrieved": retrieved_ids,
            "expected": expected_id,
            "query": query_id,
        }
        eval_results.append(eval_result)
    return eval_results


from sentence_transformers.evaluation import InformationRetrievalEvaluator
from sentence_transformers import SentenceTransformer

def evaluate_st(
    dataset,
    model_id,
    name,
):
    corpus = dataset.corpus
    queries = dataset.queries
    relevant_docs = dataset.relevant_docs

    evaluator = InformationRetrievalEvaluator(queries, corpus, relevant_docs, name=name)
    model = SentenceTransformer(model_id)
    return evaluator(model, output_path="results/")

### Eval OpenAI vs. Base Embedding vs. FineTuned Embedding

In [29]:
ada = OpenAIEmbedding()
ada_val_results = evaluate(val_dataset, ada)

df_ada = pd.DataFrame(ada_val_results)

hit_rate_ada = df_ada['is_hit'].mean()
hit_rate_ada

Generating embeddings:   0%|          | 0/84 [00:00<?, ?it/s]

  0%|          | 0/168 [00:00<?, ?it/s]

In [31]:
bge = "local:BAAI/bge-small-en"
bge_val_results = evaluate(val_dataset, bge)

df_bge = pd.DataFrame(bge_val_results)

hit_rate_bge = df_bge['is_hit'].mean()
hit_rate_bge

Generating embeddings:   0%|          | 0/84 [00:00<?, ?it/s]

  0%|          | 0/168 [00:00<?, ?it/s]

0.8511904761904762

In [32]:
finetuned = "local:test_model"
val_results_finetuned = evaluate(val_dataset, finetuned)

df_finetuned = pd.DataFrame(val_results_finetuned)

hit_rate_finetuned = df_finetuned['is_hit'].mean()
hit_rate_finetuned

Generating embeddings:   0%|          | 0/84 [00:00<?, ?it/s]

  0%|          | 0/168 [00:00<?, ?it/s]

0.8630952380952381

In [33]:
evaluate_st(val_dataset, "BAAI/bge-small-en", name='bge')
evaluate_st(val_dataset, "test_model", name='finetuned')

0.7134318346573653

# Summary

In [ ]:
df_ada['model'] = 'ada'
df_bge['model'] = 'bge'
df_finetuned['model'] = 'fine_tuned'

df_all = pd.concat([df_ada, df_bge, df_finetuned])
df_all.groupby('model').mean('is_hit')

In [35]:
df_st_bge = pd.read_csv('results/Information-Retrieval_evaluation_bge_results.csv')
df_st_finetuned = pd.read_csv('results/Information-Retrieval_evaluation_finetuned_results.csv')

df_st_bge['model'] = 'bge'
df_st_finetuned['model'] = 'fine_tuned'
df_st_all = pd.concat([df_st_bge, df_st_finetuned])
df_st_all = df_st_all.set_index('model')
df_st_all

,epoch,steps,cos_sim-Accuracy@1,cos_sim-Accuracy@3,cos_sim-Accuracy@5,cos_sim-Accuracy@10,cos_sim-Precision@1,cos_sim-Recall@1,cos_sim-Precision@3,cos_sim-Recall@3,...,dot_score-Recall@1,dot_score-Precision@3,dot_score-Recall@3,dot_score-Precision@5,dot_score-Recall@5,dot_score-Precision@10,dot_score-Recall@10,dot_score-MRR@10,dot_score-NDCG@10,dot_score-MAP@100
model,,,,,,,,,,,,,,,,,,,,,
bge,-1,-1,0.529762,0.738095,0.821429,0.898810,0.529762,0.529762,0.246032,0.738095,...,0.529762,0.246032,0.738095,0.164286,0.821429,0.089881,0.898810,0.653784,0.713158,0.658029
fine_tuned,-1,-1,0.607143,0.773810,0.863095,0.922619,0.607143,0.607143,0.257937,0.773810,...,0.607143,0.257937,0.773810,0.172619,0.863095,0.092262,0.922619,0.710249,0.761789,0.713432
